In [1]:
!git clone https://github.com/xuannianz/DifferentiableBinarization

Cloning into 'DifferentiableBinarization'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 11), reused 21 (delta 5), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [2]:
%cd /content/DifferentiableBinarization/

/content/DifferentiableBinarization


In [3]:
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-fr6u58iv
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-fr6u58iv
     |████████████████████████████████| 337kB 2.7MB/s 
     |████████████████████████████████| 30.6MB 144kB/s 
     |████████████████████████████████| 25.0MB 119kB/s 
     |████████████████████████████████| 2.1MB 45.3MB/s 
     |████████████████████████████████| 377.0MB 41kB/s 
     |████████████████████████████████| 3.2MB 37.8MB/s 
     |████████████████████████████████| 491kB 45.8MB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=46e4f9b975eef48c30e1828079e25ce41cc3cfd7542abdbe79f6cc8715c32fef
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=267009 sha256=91c6d018975729b517e0134da46d547ea62ae1c7883e9

In [4]:
!pip install pyclipper

     |████████████████████████████████| 133kB 2.6MB/s 


In [5]:
!pip uninstall -y tensorflow

Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0


In [6]:
!pip install tensorflow-gpu==1.14.0

In [7]:
!pip uninstall -y keras

Uninstalling Keras-2.2.5:
  Successfully uninstalled Keras-2.2.5


In [8]:
!pip install Keras==2.2.5

  Using cached https://files.pythonhosted.org/packages/f8/ba/2d058dcf1b85b9c212cc58264c98a4a7dd92c989b798823cc5690d062bb2/Keras-2.2.5-py2.py3-none-any.whl
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.


In [13]:
!pip install keras --upgrade

     |████████████████████████████████| 378kB 2.6MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [12]:
!pip install tensorflow --upgrade

     |████████████████████████████████| 516.2MB 33kB/s 
     |████████████████████████████████| 3.0MB 39.1MB/s 
     |████████████████████████████████| 460kB 47.2MB/s 
ERROR: tensorflow-gpu 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 2.2.2 which is incompatible.
ERROR: tensorflow-gpu 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 2.2.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0


In [2]:
%cd /content/DifferentiableBinarization/

/content/DifferentiableBinarization


In [3]:
################################################################################
#                                  inference.py                                #
################################################################################
import math
import cv2
import os.path as osp
import glob
import numpy as np
from shapely.geometry import Polygon
import pyclipper

from model import dbnet


def resize_image(image, image_short_side=736):
    height, width, _ = image.shape
    if height < width:
        new_height = image_short_side
        new_width = int(math.ceil(new_height / height * width / 32) * 32)
    else:
        new_width = image_short_side
        new_height = int(math.ceil(new_width / width * height / 32) * 32)
    resized_img = cv2.resize(image, (new_width, new_height))
    return resized_img


def box_score_fast(bitmap, _box):
    # 计算 box 包围的区域的平均得分
    h, w = bitmap.shape[:2]
    box = _box.copy()
    xmin = np.clip(np.floor(box[:, 0].min()).astype(np.int), 0, w - 1)
    xmax = np.clip(np.ceil(box[:, 0].max()).astype(np.int), 0, w - 1)
    ymin = np.clip(np.floor(box[:, 1].min()).astype(np.int), 0, h - 1)
    ymax = np.clip(np.ceil(box[:, 1].max()).astype(np.int), 0, h - 1)

    mask = np.zeros((ymax - ymin + 1, xmax - xmin + 1), dtype=np.uint8)
    box[:, 0] = box[:, 0] - xmin
    box[:, 1] = box[:, 1] - ymin
    cv2.fillPoly(mask, box.reshape(1, -1, 2).astype(np.int32), 1)
    return cv2.mean(bitmap[ymin:ymax + 1, xmin:xmax + 1], mask)[0]


def unclip(box, unclip_ratio=1.5):
    poly = Polygon(box)
    distance = poly.area * unclip_ratio / poly.length
    offset = pyclipper.PyclipperOffset()
    offset.AddPath(box, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    expanded = np.array(offset.Execute(distance))
    return expanded


def get_mini_boxes(contour):
    bounding_box = cv2.minAreaRect(contour)
    points = sorted(list(cv2.boxPoints(bounding_box)), key=lambda x: x[0])

    index_1, index_2, index_3, index_4 = 0, 1, 2, 3
    if points[1][1] > points[0][1]:
        index_1 = 0
        index_4 = 1
    else:
        index_1 = 1
        index_4 = 0
    if points[3][1] > points[2][1]:
        index_2 = 2
        index_3 = 3
    else:
        index_2 = 3
        index_3 = 2

    box = [points[index_1], points[index_2],
           points[index_3], points[index_4]]
    return box, min(bounding_box[1])


def polygons_from_bitmap(pred, bitmap, dest_width, dest_height, max_candidates=100, box_thresh=0.7):
    pred = pred[..., 0]
    bitmap = bitmap[..., 0]
    height, width = bitmap.shape
    boxes = []
    scores = []

    _, contours, _ = cv2.findContours((bitmap * 255).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours[:max_candidates]:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        points = approx.reshape((-1, 2))
        if points.shape[0] < 4:
            continue
        score = box_score_fast(pred, points.reshape(-1, 2))
        if box_thresh > score:
            continue
        if points.shape[0] > 2:
            box = unclip(points, unclip_ratio=2.0)
            if len(box) > 1:
                continue
        else:
            continue
        box = box.reshape(-1, 2)
        _, sside = get_mini_boxes(box.reshape((-1, 1, 2)))
        if sside < 5:
            continue

        box[:, 0] = np.clip(np.round(box[:, 0] / width * dest_width), 0, dest_width)
        box[:, 1] = np.clip(np.round(box[:, 1] / height * dest_height), 0, dest_height)
        boxes.append(box.tolist())
        scores.append(score)
    return boxes, scores


def foo():
    mean = np.array([103.939, 116.779, 123.68])
    count = 0;
    _, model = dbnet()
    model.load_weights('db_48_2.0216_2.5701.h5', by_name=True, skip_mismatch=True)
    for image_path in glob.glob(osp.join('datasets', '*.jpg')):
        count += 1
        print("\r count", end="")
        image = cv2.imread(image_path)
        src_image = image.copy()
        h, w = image.shape[:2]
        image = resize_image(image)
        image = image.astype(np.float32)
        image -= mean
        image_input = np.expand_dims(image, axis=0)
        p = model.predict(image_input)[0]
        bitmap = p > 0.3
        boxes, scores = polygons_from_bitmap(p, bitmap, w, h, box_thresh=0.5)
        for box in boxes:
            cv2.drawContours(src_image, [np.array(box)], -1, (0, 255, 0), 2)
        cv2.namedWindow('image', cv2.WINDOW_NORMAL)
        cv2.imshow('image', src_image)
        cv2.waitKey(0)
        image_fname = osp.split(image_path)[-1]
        cv2.imwrite('test/' + image_fname, src_image)

Using TensorFlow backend.


In [10]:
%cd ..

/content


In [12]:
mean = np.array([103.939, 116.779, 123.68])
count = 0;
_, model = dbnet()
model.load_weights('db_48_2.0216_2.5701.h5', by_name=True, skip_mismatch=True)

KeyError: ignored

In [7]:
!cp -r /content/drive/'My Drive'/datasets/* /content/DifferentiableBinarization/datasets/

^C


In [0]:
from google.colab.patches import cv2_imshow
cv2.imgshow = cv2_imshow

for image_path in glob.glob(osp.join('datasets/TD_TR/TD500/test_images', '*.JPG')):
    count += 1
    print(f"\r {count}", end="")
    image = cv2.imread(image_path)
    src_image = image.copy()
    h, w = image.shape[:2]
    image = resize_image(image)
    image = image.astype(np.float32)
    image -= mean
    image_input = np.expand_dims(image, axis=0)
    p = model.predict(image_input)[0]
    bitmap = p > 0.5
    boxes, scores = polygons_from_bitmap(p, bitmap, w, h, box_thresh=0.5)
    for box in boxes:
        cv2.drawContours(src_image, [np.array(box)], -1, (0, 255, 0), 2)
    #cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2_imshow(src_image)
    image_fname = osp.split(image_path)[-1]
    cv2.imwrite('test/' + image_fname, src_image)
    

In [14]:
%cd /content/DifferentiableBinarization/

/content/DifferentiableBinarization


In [17]:
import datetime
import os.path as osp
from keras import callbacks
from keras import optimizers
from keras.utils import get_file
import os

from generator import generate
from model import dbnet
checkpoints_dir = f'checkpoints/{datetime.date.today()}'

batch_size = 16

if not osp.exists(checkpoints_dir):
    os.makedirs(checkpoints_dir)

train_generator = generate('datasets/icdar2015', batch_size=batch_size, is_training=True)
val_generator = generate('datasets/icdar2015', batch_size=batch_size, is_training=False)

model, prediction_model = dbnet()
resnet_filename = 'ResNet-50-model.keras.h5'
resnet_resource = 'https://github.com/fizyr/keras-models/releases/download/v0.0.1/{}'.format(resnet_filename)
resnet_filepath = get_file(resnet_filename, resnet_resource, cache_subdir='models',
                           md5_hash='3e9f4e4f77bbe2c9bec13b53ee1c2319')
model.load_weights(resnet_filepath, by_name=True, skip_mismatch=True)
model.compile(optimizer=optimizers.Adam(lr=1e-3), loss={'db_loss': lambda y_true, y_pred: y_pred})
checkpoint = callbacks.ModelCheckpoint(
    osp.join(checkpoints_dir, 'db_{epoch:02d}_{loss:.4f}_{val_loss:.4f}.h5'),
    verbose=1,
)
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=200,
    initial_epoch=0,
    epochs=100,
    verbose=1,
    callbacks=[checkpoint],
    validation_data=val_generator,
    validation_steps=19
)

Epoch 1/100


FileNotFoundError: ignored